In [ ]:
!pip install keras-tuner


     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=341277625bd0d8d45bd2cbe67ff6ecade3624d569f60e0e89f12a55528e87a5f
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=27ac30c1896b677b24dd4aa79b9ff7d9b0c1290adf62063be405c831455aad7a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [69]:
def build_model(hp):
   model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)   
     ),
     keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'

     ),
     keras.layers.Flatten(),
     keras.layers.Dense(
         units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
         activation='relu'
     ),
     keras.layers.Dense(10,activation='softmax') #output layer   
   ])
   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
   return model

In [70]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [71]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 34m 50s]
val_accuracy: 0.9079999923706055

Best val_accuracy So Far: 0.9098333120346069
Total elapsed time: 01h 01m 02s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |128               
conv_1_kernel     |5                 |5                 
conv_2_filter     |80                |48                
conv_2_kernel     |3                 |3                 
dense_1_units     |64                |64                
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 292s 173ms/step - loss: 0.4079 - accuracy: 0.8514 - val_loss: 0.3072 - val_accuracy: 0.8863
Epoch 2/3
1688/1688 [==============================] - 292s 173ms/step - loss: 0.2687 - accuracy: 0.9014 - val_loss: 0.2873 - val_accuracy: 0.8982
Epoch 3/3
 182/1688 [==>...........................] - ETA: 4:15 - loss: 0.2024 - accuracy: 0.9243

In [ ]:
model=tuner_search.get_best_models(num_model=1)[0]

In [ ]:
model.summary()